In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
from rateFunction import *
spark = SparkSession.builder.appName("Ngram_buzzwords").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [ ]:
df = spark.read.csv('../../data/eng-gb-all-only-letters.csv', sep="\t", header=False)

In [ ]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [ ]:
#df.count()

In [ ]:
rdd = df.rdd.map(list)

In [ ]:
def splitAndMakeNgramKey(row):
    elements = row[0].split(",")
    return(elements[0], (elements[1:len(elements)]))

rdd = rdd.map(splitAndMakeNgramKey)

In [ ]:
#rdd.take(10)

In [ ]:
#rdd2 = sc.parallelize(rdd.take(1000000))

In [ ]:
def groupData(a,b):
    if isinstance(a[0], (list,)):
        return a+[b]
    else:
        return [a]+[b]

rdd3 = rdd.reduceByKey(groupData)

In [ ]:
rdd3.take(10)

In [ ]:
rdd3.coalesce(1).saveAsTextFile("Words-with-year-list.txt")